<a href="https://colab.research.google.com/github/unknown-jun/NLP_study/blob/main/NLP_Book/B08_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 임베딩
- **<span style="color: 2D3748; background-color:#fff5b1;">임베딩(embedding):</br>** 사람이 사용하는 언어(자연어)를 컴퓨터가 이해할 수 있는 언어(숫자) 형태인 벡터(vector)로 변환한 결과 혹은 일련의 과정을 의미함  

임베딩의 역할
- 단어 및 문장 간 관련성 계산
- 의미적 혹은 문법적 정보의 함축(왕-여왕, 교사-학생)



## 희소 표현 기반 임베딩
- **<span style="color: 2D3748; background-color:#fff5b1;">희소 표현(sparse representation)</span>**은 대부분 값이 0으로 채워져 있는 경우로, 대표적으로 **<span style="color: 2D3748; background-color:#fff5b1;">원-핫 인코딩</span>**이 있다.  
- 단어 $N$개를 각각 $N$차원의 벡터로 표현하는 방식으로 단어가 포함되어 있는 위치에 1을 넣고 나머지에는 0 값을 채워넣는 방식.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

class2 = pd.read_csv('https://raw.githubusercontent.com/gilbutITbook/080289/main/chap10/data/class2.csv')
label_encoding = LabelEncoder()
onehot_encoder = OneHotEncoder()

train_x = label_encoding.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

원-핫인코딩의 단점
1. 수학적인 의미에서 원-핫 벡터들은 하나의 요소만 1을 갖고 나머지는 모두 0인 **<span style="color: 2D3748; background-color:#fff5b1;">희소 벡터(sparse vector)</span>**를 갖는다.
  - 이때 두 단어에 대한 벡터의 내적(inner product)을 구해보면 0 값을 갖게 되므로 **<span style="color: 2D3748; background-color:#fff5b1;">직교(orthogonal)</span>**를 이룬다.
  - 즉, 단어끼리 관계성(유의어, 반의어)없이 서로 **<span style="color: 2D3748; background-color:#fff5b1;">독립적인</span>** 관계를 가진다.
2. **<span style="color: 2D3748; background-color:#fff5b1;">차원의 저주(curse of dimentionality)</span>** 문제가 발생한다.  

원-핫 인코딩에 대한 대안으로 워드투벡터(Word2Vec), 글로브(GloVe), 패스트텍스트(FastText)와 같이 신경망에 기반하여 단어를 벡터로 바꾸는 방법론이 주목받고 있다.

## 횟수 기반 임베딩
횟수 기반은 출현한 빈도를 고려하여 임베딩을 하는 방법, 대표적인 방식으로 TF-IDF가 있다.

### 카운터 벡터
- **<span style="color: 2D3748; background-color:#fff5b1;">카운터 벡터(counter vector)</span>**는 문자 집합에서 단어를 토큰으로 생성하고 각 단어의 출현 빈도수를 이용하여 인코딩해서 벡터로 만드는 방법
- 카운터 벡터는 사이킷런의 `CountVectorize()`를 사용하여 구현 가능하며 다음과 같은 작업을 수행한다.
  1. 문서를 토큰 리스트로 변환
  2. 각 문서에서 토큰의 출현 빈도를 센다.
  3. 각 문서를 인코딩하고 벡터로 변환

In [2]:
# 코퍼스에서 카운터 벡터 적용
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]
vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'and': 0,
 'any': 1,
 'chance': 2,
 'do': 3,
 'get': 4,
 'have': 5,
 'if': 6,
 'is': 7,
 'last': 8,
 'never': 9,
 'not': 10,
 'one': 11,
 'please': 12,
 'this': 13,
 'will': 14,
 'you': 15}

In [3]:
# 배열 변환
vect.transform(['you will never get chance, so do not miss your chance']).toarray()

array([[0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1]])

In [4]:
# 불용어를 제거한 카운터 벡터
vect = CountVectorizer(stop_words=['and', 'is', 'please', 'this']).fit(corpus)
vect.vocabulary_

{'any': 0,
 'chance': 1,
 'do': 2,
 'get': 3,
 'have': 4,
 'if': 5,
 'last': 6,
 'never': 7,
 'not': 8,
 'one': 9,
 'will': 10,
 'you': 11}

### TF-IDF
**<span style="color: 2D3748; background-color:#fff5b1;">TF-IDF(Term Frequency-Inverse Document Frequency)</span>**는 정보 검색론에서 가중치를 구할때 사용되는 알고리즘  
- **<span style="color: 2D3748; background-color:#fff5b1;">TF(Term Frequence)</span>**은 문서 내에서 특정 단어가 출현한 빈도
  - '딥러닝 이대로 괜찮은가?'라는 기사에 "딥러닝"이라는 단어가 15번 출현했다면 이 문서의 TF는 15이다.
- **<span style="color: 2D3748; background-color:#fff5b1;">DF(Document Frequence)</span>**은 한 단어가 전체 문서에서 얼마나 공통적으로 등장하는지를 나타내는 값. 즉, 특정 단어가 나타난 문서 개수
  - 10개의 기사글에 '딥러닝'이라는 단어가 나온 기사는 2개다. 
  - '딥러닝 이대로 괜찮은가?'에서 15번 나왔지만 그것은 중요하지 않고 전체 기사글에서 '딥러닝'이라는 단어가 나온 기사 글의 갯수가 중요하다.
- 특정 단어 $t$가 모든 문서에 등장하는 일반적인 단어('a', 'the')라면 그것이 주는 정보량은 많지 않으므로 가중치를 낮추어 줄 필요가 있다.
- 따라서 DF가 클수록 TF-IDF의 가중치 값을 줄이기 위해 DF값에 역수를 취하는데 이 값이 **<span style="color: 2D3748; background-color:#fff5b1;">IDF</span>**이다.
- 역수를 취할 경우 전체 문서 개수가 많아질수록 IDF 값도 커지므로 로그$log$를 취해야 한다.
$$idf_t = log \frac{N}{df_t} = log\frac{\text{전체 문서 개수}}{\text{특정 단어 t가 포함된 문서 개수}}$$
- 전체 문서에서 특정 단어가 발생하는 빈도가 0이라면 분모가 0이 되기 때문에 이를 방지하기 위해 분모에 1을 더해준다. 이를 스무딩(smoothing)이라고 한다.
$$idf_t = log \frac{N}{1+ df_t } = log\frac{\text{전체 문서 개수}}{1+\text{특정 단어 t가 포함된 문서 개수 }}$$

TF-IDF는 다음과 같은 상황에서 사용한다.
- 키워드 검색을 기반으로 하는 검색 엔진
- 중요 키워드 분석
- 검색 엔진에서 검색 결과의 순위를 결정